In [1]:
# Importing required libraries 
# Keras
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Other  
import librosa
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import os
import pickle
import wave
import IPython.display as ipd  # To play sound in the notebook
import python_speech_features as ps
import seaborn as sns
import scipy
import math
import time
from scipy import signal
import sys

In [2]:
def trimVoice(data, sokuonleng = 3, spaceleng = 10, voiceleng= 10, debug=False):
    trimpoint = detectVoice(data, sokuonleng = sokuonleng, spaceleng=spaceleng, voiceleng = voiceleng, debug=debug)
    trimdata = []
    for point in trimpoint:
        trimdata.append(data[point[0]:point[1]])
    return trimdata

In [3]:
def detectVoice(data, sokuonleng = 3, spaceleng = 10, voiceleng= 10, debug=False):
    zscore = np.empty_like(data)
    eps = 1e-5
    mean = np.mean(data)
    std = np.std(data)
    zscore = (data - mean)/(std+eps)
    zscore = np.append(zscore, np.zeros(1024 - zscore.shape[0] % 1024),0)
    zscore = zscore.reshape(-1, 1024)
    ave = np.empty((1,), float)
    diff = np.empty((0,), float)
    silent = 0
    voice = 0
    isVoice = 0
    standby = 0
    voiceOnWave = np.empty((0,2), int)
    warn = False
    for waddr in range(zscore.shape[0]):
        ave = np.append(ave, np.mean(np.abs(zscore[waddr])))
        diff = np.append(diff, ave[-2]-ave[-1])
        #print(np.abs(diff[waddr]) > 0)
        #声の部分か判定する。
        if np.abs(diff[waddr]) > 0.05:
            silent = 0
            voice  = sokuonleng
        else:
            silent += 1
            voice  -= 1 if voice > 0 else 0
        #発話していないなら
        if isVoice == 0:
            if standby == -1:
                #スタンバイでない時、音を検知したら、スタンバイする(その時の時間を覚えておく)。
                if voice == sokuonleng:
                    standby = waddr - 1 #発声する瞬間も大事な特微量なので、現在のアドレスより1コマ過去のものから判定開始
            else:
                if waddr-standby >= voiceleng:
                    #voiceleng以上発話していれば、isRecをアクティブに。
                    if voice != 0:
                        if debug == True:
                            print("Record start{}".format(standby, waddr))
                        isVoice = 1
                        voiceOnWave = np.append(voiceOnWave, np.zeros((1,2), int),0)
                        voiceOnWave[-1,0] = standby*1024
                    #voicelengより短い発話だったら、スタンバイ解除
                    elif voice == 0:
                        standby = -1
                elif silent >= spaceleng:
                    standby = -1
                
        #発話しているなら
        else:
            #もしも、一定時間以上静寂なら、isRecをディスアクティブに。
            if silent >= spaceleng or waddr == zscore.shape[0]-1:
                if debug == True:
                    print("--->{}".format(waddr-spaceleng, waddr))
                isVoice = 0
                standby = -1
                voiceOnWave[-1,1] = (waddr - silent +5)*1024 #+5して発話後に若干余白をとる。
    return voiceOnWave

In [4]:
def loadwav(path, mono=True):
    file = wave.open(path, 'r')
    params = file.getparams()
    nchannels, samplewidth, samplerate, wav_length = params[:4]
    datatype = {2:"int16", 4:"int32"}
    str_data = file.readframes(wav_length)
    data = np.frombuffer(str_data, dtype = np.short)
    if(nchannels == 2):
        left = data[::2]
        right= data[1::2]
        wavedata = np.stack([left, right], 1)
        if mono == True:
            wavedata = np.mean(wavedata, axis=1)
            wavedata = wavedata.astype(datatype[samplewidth])
    else:
        wavedata = data
    return wavedata, samplerate, samplewidth, nchannels

In [5]:
def savewav(data, path):
    outd = struct.pack("h" * len(data), *data)
    with wave.open(path, 'w') as ww:
        ww.setnchannels(nchannel)
        ww.setsampwidth(samplewidt)
        ww.setframerate(samplerate)
        ww.writeframes(outd)

In [6]:
def mellCepstrum(data, RATE, winlen=0.08, winstep=0.016, nfilt=40, start=None, end=None):
    data = np.append(data, np.zeros(1024 - data.shape[0] % 1024),0)
    eps = 1e-5
    if start == None and end == None:
        mel_spec = ps.logfbank(data, samplerate=RATE, winlen=winlen, winstep=winstep, nfilt=nfilt, nfft=int(winlen*RATE))
    elif start == None or end == None:
        point = start if end == None else end
        mel_spec = ps.logfbank(data[point], samplerate=RATE, winlen=winlen, winstep=winstep, nfilt=nfilt, nfft=int(winlen*RATE))
    else:
        mel_spec = ps.logfbank(data[start:end], samplerate=RATE, winlen=winlen, winstep=winstep, nfilt=nfilt, nfft=int(winlen*RATE))
    #deltaの特微量を生成
    delta1= ps.delta(mel_spec, 4)
    #delta-deltaの特微量を生成
    delta2 = ps.delta(delta1, 4)
    mean1 = np.mean(mel_spec)
    mean2 = np.mean(delta1)
    mean3 = np.mean(delta2)
    std1 = np.std(mel_spec)
    std2 = np.std(delta1)
    std3 = np.std(delta2)
    procd = np.empty((mel_spec.shape[0], nfilt, 3))
    procd[:,:,0] = (mel_spec - mean1)/(std1+eps) #mel_spec
    procd[:,:,1] = (delta1 - mean2)/(std2+eps) #delta1
    procd[:,:,2] = (delta2 - mean3)/(std3+eps) #delta2
    return procd

In [7]:
def inverselabel(value):
    tag = {'female_angry':0, 'female_disgust':1, 'female_fear':2, 'female_happy':3,
 'female_neutral':4, 'female_sad':5, 'female_surprise':6, 'male_angry':7,
 'male_disgust':8, 'male_fear':9, 'male_happy':10, 'male_neutral':11, 'male_sad':12,
 'male_surprise':13}
    return [k for k, v in tag.items() if v == value][0]

In [8]:
# loading json and model architecture 
json_file = open('model_json.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("saved_models/Emotion_Model.h5")
print("Loaded model from disk")

Loaded model from disk


In [13]:
# lets pick up the meta-data that we got from our first part of the Kernel
ref = pd.read_csv("Data_path.csv")
ref.head()

,labels,source,path
0,male_happy,SAVEE,/home/naokitakatani/datasets/SAVEE/DC_h03.wav
1,male_fear,SAVEE,/home/naokitakatani/datasets/SAVEE/JE_f12.wav
2,male_happy,SAVEE,/home/naokitakatani/datasets/SAVEE/DC_h02.wav
3,male_neutral,SAVEE,/home/naokitakatani/datasets/SAVEE/KL_n01.wav
4,male_happy,SAVEE,/home/naokitakatani/datasets/SAVEE/DC_h07.wav


In [14]:
ref_s = ref.sample(frac=1)
ref_s.head()

,labels,source,path
3370,female_angry,TESS,/home/naokitakatani/datasets/TESS/YAF_angry/YA...
700,female_fear,RAVDESS,/home/naokitakatani/datasets/RAVDESS/Actor_04/...
7686,female_sad,CREMA,/home/naokitakatani/datasets/CREMA-D/1037_ITH_...
87,male_disgust,SAVEE,/home/naokitakatani/datasets/SAVEE/JE_d13.wav
4724,male_neutral,CREMA,/home/naokitakatani/datasets/CREMA-D/1001_DFA_...


In [15]:
b, a = signal.iirfilter(2, 1500,btype='highpass',rs = 60,rp = 1.0, ftype='ellip',output='ba',fs=44100)
accuracy = np.empty(0)
for index, (label, path) in enumerate(zip(ref_s.labels, ref_s.path)):
    print("No.{} path : {}".format(index, path))
    try:
        X, samplerate,samplewidth,nchannel = loadwav(path)
        trimdata = trimVoice(X, spaceleng = 20)
        if len(trimdata)>1:
            print("Info : Extracted multiple({}) part.".format(len(trimdata)))
        for data in trimdata:
            Y = signal.lfilter(b, a, data)
            heatmap_data = mellCepstrum(Y, samplerate)
            if heatmap_data.shape[0] <= 300:
                feature = np.vstack((heatmap_data, np.zeros((300-heatmap_data.shape[0],40,3),float)))
            else :
                feature = heatmap_data[0:300]
            preds = loaded_model.predict_step(np.expand_dims(feature,axis=0))
            preds = preds.numpy()
            print("prdict label : {}".format(inverselabel(preds.argmax())))
            print("actual label : {}".format(label))
            if inverselabel(preds.argmax()) == label:
                accuracy = np.append(accuracy, 1)
            else:s
                accuracy = np.append(accuracy, 0)
            print("acc_rate : {}".format(np.sum(accuracy)/accuracy.shape[0]))
    except KeyboardInterrupt:
        print("KeyboardInterrupt")
        sys.exit()
    except:
        print("Skip this audio file.")

No.0 path : /home/naokitakatani/datasets/TESS/YAF_angry/YAF_search_angry.wav
prdict label : female_angry
actual label : female_angry
acc_rate : 1.0
No.1 path : /home/naokitakatani/datasets/RAVDESS/Actor_04/03-01-06-01-01-01-04.wav
prdict label : female_angry
actual label : female_fear
acc_rate : 0.5
No.2 path : /home/naokitakatani/datasets/CREMA-D/1037_ITH_SAD_XX.wav
prdict label : female_disgust
actual label : female_sad
acc_rate : 0.3333333333333333
No.3 path : /home/naokitakatani/datasets/SAVEE/JE_d13.wav
prdict label : male_angry
actual label : male_disgust
acc_rate : 0.25
No.4 path : /home/naokitakatani/datasets/CREMA-D/1001_DFA_NEU_XX.wav
prdict label : male_neutral
actual label : male_neutral
acc_rate : 0.4
No.5 path : /home/naokitakatani/datasets/CREMA-D/1035_IWW_NEU_XX.wav
prdict label : male_neutral
actual label : male_neutral
acc_rate : 0.5
No.6 path : /home/naokitakatani/datasets/RAVDESS/Actor_11/03-01-05-02-01-01-11.wav
prdict label : male_angry
actual label : male_angry
a

prdict label : female_surprise
actual label : female_surprise
acc_rate : 0.7017543859649122
No.57 path : /home/naokitakatani/datasets/CREMA-D/1062_ITS_HAP_XX.wav
prdict label : male_angry
actual label : male_happy
acc_rate : 0.6896551724137931
No.58 path : /home/naokitakatani/datasets/RAVDESS/Actor_13/03-01-07-01-01-01-13.wav
prdict label : female_angry
actual label : male_disgust
acc_rate : 0.6779661016949152
No.59 path : /home/naokitakatani/datasets/CREMA-D/1046_ITS_NEU_XX.wav
prdict label : female_neutral
actual label : female_neutral
acc_rate : 0.6833333333333333
No.60 path : /home/naokitakatani/datasets/CREMA-D/1040_IWL_NEU_XX.wav
prdict label : male_neutral
actual label : male_neutral
acc_rate : 0.6885245901639344
No.61 path : /home/naokitakatani/datasets/CREMA-D/1063_ITH_DIS_XX.wav
prdict label : female_disgust
actual label : female_disgust
acc_rate : 0.6935483870967742
No.62 path : /home/naokitakatani/datasets/CREMA-D/1053_TIE_HAP_XX.wav
prdict label : female_happy
actual label

prdict label : male_disgust
actual label : male_disgust
acc_rate : 0.6548672566371682
No.112 path : /home/naokitakatani/datasets/CREMA-D/1012_MTI_DIS_XX.wav
prdict label : female_disgust
actual label : female_disgust
acc_rate : 0.6578947368421053
No.113 path : /home/naokitakatani/datasets/RAVDESS/Actor_02/03-01-04-02-01-01-02.wav
prdict label : female_sad
actual label : female_sad
acc_rate : 0.6608695652173913
No.114 path : /home/naokitakatani/datasets/CREMA-D/1016_IEO_FEA_LO.wav
prdict label : male_disgust
actual label : male_fear
acc_rate : 0.6551724137931034
No.115 path : /home/naokitakatani/datasets/CREMA-D/1048_ITS_HAP_XX.wav
prdict label : male_angry
actual label : male_happy
acc_rate : 0.6495726495726496
No.116 path : /home/naokitakatani/datasets/CREMA-D/1064_IEO_SAD_LO.wav
prdict label : male_disgust
actual label : male_sad
acc_rate : 0.6440677966101694
No.117 path : /home/naokitakatani/datasets/RAVDESS/Actor_13/03-01-02-02-01-01-13.wav
prdict label : male_neutral
actual label 

prdict label : female_angry
actual label : female_angry
acc_rate : 0.6882352941176471
No.169 path : /home/naokitakatani/datasets/RAVDESS/Actor_21/03-01-06-02-02-02-21.wav
prdict label : male_fear
actual label : male_fear
acc_rate : 0.6900584795321637
No.170 path : /home/naokitakatani/datasets/CREMA-D/1019_ITS_FEA_XX.wav
prdict label : male_disgust
actual label : male_fear
acc_rate : 0.686046511627907
No.171 path : /home/naokitakatani/datasets/CREMA-D/1005_DFA_DIS_XX.wav
prdict label : male_disgust
actual label : male_disgust
acc_rate : 0.6878612716763006
No.172 path : /home/naokitakatani/datasets/CREMA-D/1035_WSI_HAP_XX.wav
prdict label : male_happy
actual label : male_happy
acc_rate : 0.6896551724137931
No.173 path : /home/naokitakatani/datasets/CREMA-D/1070_TIE_FEA_XX.wav
prdict label : male_angry
actual label : male_fear
acc_rate : 0.6857142857142857
No.174 path : /home/naokitakatani/datasets/CREMA-D/1039_MTI_ANG_XX.wav
prdict label : male_angry
actual label : male_angry
acc_rate : 

prdict label : female_angry
actual label : female_angry
acc_rate : 0.7149122807017544
No.227 path : /home/naokitakatani/datasets/CREMA-D/1070_IOM_HAP_XX.wav
prdict label : male_angry
actual label : male_happy
acc_rate : 0.7117903930131004
No.228 path : /home/naokitakatani/datasets/CREMA-D/1007_ITH_DIS_XX.wav
prdict label : female_disgust
actual label : female_disgust
acc_rate : 0.7130434782608696
No.229 path : /home/naokitakatani/datasets/TESS/YAF_disgust/YAF_pad_disgust.wav
prdict label : female_disgust
actual label : female_disgust
acc_rate : 0.7142857142857143
No.230 path : /home/naokitakatani/datasets/CREMA-D/1019_ITS_DIS_XX.wav
prdict label : male_disgust
actual label : male_disgust
acc_rate : 0.7155172413793104
No.231 path : /home/naokitakatani/datasets/RAVDESS/Actor_22/03-01-03-02-01-01-22.wav
prdict label : female_angry
actual label : female_happy
acc_rate : 0.7124463519313304
No.232 path : /home/naokitakatani/datasets/CREMA-D/1036_ITS_FEA_XX.wav
prdict label : male_disgust
act

...

prdict label : female_sad
actual label : female_sad
acc_rate : 0.7387612387612388
No.1995 path : /home/naokitakatani/datasets/CREMA-D/1030_IWL_NEU_XX.wav
prdict label : female_neutral
actual label : female_neutral
acc_rate : 0.7388916625062406
No.1996 path : /home/naokitakatani/datasets/CREMA-D/1036_IEO_FEA_MD.wav
prdict label : male_fear
actual label : male_fear
acc_rate : 0.7390219560878244
No.1997 path : /home/naokitakatani/datasets/TESS/YAF_pleasant_surprised/YAF_mop_ps.wav
prdict label : female_surprise
actual label : female_surprise
acc_rate : 0.7391521197007481
No.1998 path : /home/naokitakatani/datasets/CREMA-D/1020_TSI_ANG_XX.wav
prdict label : female_angry
actual label : female_angry
acc_rate : 0.7392821535393819
No.1999 path : /home/naokitakatani/datasets/TESS/OAF_angry/OAF_merge_angry.wav
prdict label : female_angry
actual label : female_angry
acc_rate : 0.739412057797708
No.2000 path : /home/naokitakatani/datasets/CREMA-D/1037_TAI_FEA_XX.wav
prdict label : male_disgust
act

prdict label : female_disgust
actual label : female_fear
acc_rate : 0.7365094798249878
No.2050 path : /home/naokitakatani/datasets/TESS/OAF_Sad/OAF_door_sad.wav
prdict label : female_disgust
actual label : female_sad
acc_rate : 0.7361516034985423
No.2051 path : /home/naokitakatani/datasets/CREMA-D/1086_IWL_SAD_XX.wav
prdict label : male_disgust
actual label : male_sad
acc_rate : 0.7357940747935892
No.2052 path : /home/naokitakatani/datasets/CREMA-D/1012_ITS_HAP_XX.wav
prdict label : female_disgust
actual label : female_happy
acc_rate : 0.7354368932038835
No.2053 path : /home/naokitakatani/datasets/CREMA-D/1048_DFA_SAD_XX.wav
prdict label : male_disgust
actual label : male_sad
acc_rate : 0.735080058224163
No.2054 path : /home/naokitakatani/datasets/CREMA-D/1040_ITS_DIS_XX.wav
prdict label : male_disgust
actual label : male_disgust
acc_rate : 0.7352085354025218
No.2055 path : /home/naokitakatani/datasets/RAVDESS/Actor_24/03-01-07-01-02-02-24.wav
prdict label : female_disgust
actual label

prdict label : female_disgust
actual label : female_disgust
acc_rate : 0.7353219696969697
No.2105 path : /home/naokitakatani/datasets/CREMA-D/1053_DFA_DIS_XX.wav
prdict label : female_disgust
actual label : female_disgust
acc_rate : 0.735447231424515
No.2106 path : /home/naokitakatani/datasets/CREMA-D/1021_IEO_ANG_MD.wav
prdict label : female_angry
actual label : female_angry
acc_rate : 0.7355723746452223
No.2107 path : /home/naokitakatani/datasets/CREMA-D/1091_ITH_SAD_XX.wav
prdict label : female_disgust
actual label : female_sad
acc_rate : 0.735224586288416
No.2108 path : /home/naokitakatani/datasets/CREMA-D/1038_IEO_NEU_XX.wav
prdict label : male_neutral
actual label : male_neutral
acc_rate : 0.7353497164461248
No.2109 path : /home/naokitakatani/datasets/CREMA-D/1076_DFA_ANG_XX.wav
prdict label : female_angry
actual label : female_angry
acc_rate : 0.73547472838923
No.2110 path : /home/naokitakatani/datasets/CREMA-D/1033_ITH_HAP_XX.wav
prdict label : male_happy
actual label : male_ha

prdict label : female_angry
actual label : female_neutral
acc_rate : 0.7343173431734318
No.2161 path : /home/naokitakatani/datasets/CREMA-D/1049_ITS_SAD_XX.wav
prdict label : female_sad
actual label : female_sad
acc_rate : 0.7344398340248963
No.2162 path : /home/naokitakatani/datasets/TESS/OAF_angry/OAF_dead_angry.wav
prdict label : female_angry
actual label : female_angry
acc_rate : 0.7345622119815668
No.2163 path : /home/naokitakatani/datasets/RAVDESS/Actor_11/03-01-05-01-01-01-11.wav
prdict label : male_angry
actual label : male_angry
acc_rate : 0.7346844771994473
No.2164 path : /home/naokitakatani/datasets/CREMA-D/1047_TAI_FEA_XX.wav
prdict label : female_fear
actual label : female_fear
acc_rate : 0.7348066298342542
No.2165 path : /home/naokitakatani/datasets/CREMA-D/1078_IEO_SAD_HI.wav
prdict label : female_sad
actual label : female_sad
acc_rate : 0.7349286700414174
No.2166 path : /home/naokitakatani/datasets/CREMA-D/1047_IEO_SAD_LO.wav
prdict label : female_sad
actual label : fem

Error: file does not start with RIFF id